In [1]:
import os
from openai import OpenAI

key = ""
os.environ['OPENAI_API_KEY'] = key

In [2]:
# check
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

answer = llm.invoke(question)

# 질의
print(f"## 질문: {question}")
print(f"## 답변\n{answer}")

## 질문: 대한민국의 수도는 어디인가요?
## 답변
content='대한민국의 수도는 서울입니다.' response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 24, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8f483189-dda7-45e4-93a3-b161e5058485-0' usage_metadata={'input_tokens': 24, 'output_tokens': 15, 'total_tokens': 39}


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

# 단계 1: 문서 로드(Load Documents)
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
url = "https://n.news.naver.com/mnews/article/584/0000026285"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
print(docs)

print('#'*70)
print('## chunking ##')

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
for tmp in splits:
    print(f'## {tmp}\n')

[Document(page_content="\nEU, 세계 최초 'AI 규제법' 통과…위반시 매출 7%까지 과징금\n\n\n\n\n\n\n유럽연합(EU)이 세계 최초로 인공지능(AI)를 규제하는 법을 통과시켰다. 게티이미지뱅크 제공   유럽연합(EU)이 세계 최초로 인공지능(AI)을 규제하는 법을 통과시켰다.    13일(현지시간) 유럽의회는 프랑스 스트라스부르에서 열린 본회의에서 'AI 법(AI Act)' 최종안을 찬성 523표, 반대 46표로 가결했다. 기권은 49표였다. EU는 이 법안이 빠른 속도로 발전하는 기술의 위험으로부터 시민을 보호하는 동시에 혁신을 촉진할 것이라고 보고 있다.    법안에 따르면 EU는 AI 활용 분야를 총 4단계의 위험 등급으로 나눠 차등 규제한다. 고위험 등급으로 분류되는 의료, 교육을 비롯한 공공 서비스나 선거, 핵심 인프라, 자율주행 등에서는 AI 기술 사용 시 사람이 반드시 감독하도록 하고 위험관리시스템을 구축해야 한다.   또 광범위한 사이버 공격, ‘유해한 선입견’ 전파 등 EU가 시스템적 위험이라고 규정한 사고 발생을 방지하기 위한 조치를 해야 한다. 별도의 정보 공개·고지 의무도 부과된다.   사람과 유사한 수준 또는 그 이상의 지능을 갖춘 AI인 '범용 AI'를 개발하는 기업에 대해서는 기술 개발 과정에서의 ‘투명성 의무’를 부여하기로 했다. 이 조항은 2021년 발의된 초안에는 없었지만 이듬해 챗GPT 등 생성형 AI가 보편화되면서 AI 오남용에 대한 우려가 확산돼 입법 과정에서 추가됐다.   일부 AI 기술 활용은 원천 금지된다. 이른바 개인의 특성·행동과 관련된 데이터로 개별 점수를 매기는 관행인 ‘사회적 점수 평가’(소셜 스코어링)가 대표적이다. AI를 활용한 실시간 원격 생체인식 식별 시스템 사용도 중대 범죄 용의자 수색을 비롯해 예외적인 경우를 제외하곤 사실상 금지된다. 이 밖에 딥페이크 영상이나 이미지는 AI로 만든 조작 콘텐츠라는 점을 표기하도록 했다.   이번에 통과된 법안은 EU 27개국 장관

In [4]:
!pip install langchainhub

In [5]:
from langchain import hub
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt")

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "ai 규제법에 대해 알려줘"
response = rag_chain.invoke(question)

# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

URL: https://n.news.naver.com/mnews/article/584/0000026285
문서의 수: 1
[HUMAN]
ai 규제법에 대해 알려줘

[AI]
유럽연합(EU)은 세계 최초로 인공지능(AI)를 규제하는 법을 통과시켰다. AI 규제법은 AI 활용 분야를 4단계의 위험 등급으로 나누어 차등 규제하며, 고위험 등급에서는 사람 감독과 위험관리시스템 구축이 요구된다. 법을 위반할 경우 최대 7%까지의 과징금이 부과될 수 있다.


## ChatPromptTemplate

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 텍스트로부터 FAISS 벡터 저장소를 생성합니다.
vectorstore = FAISS.from_texts(
    [
        "테디는 랭체인 주식회사에서 근무를 하였습니다.",
        "셜리는 테디와 같은 회사에서 근무하였습니다.",
        "테디의 직업은 개발자입니다.",
        "셜리의 직업은 디자이너입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)
# 벡터 저장소를 검색기로 사용합니다.
retriever = vectorstore.as_retriever()
# 템플릿을 정의합니다.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
# 템플릿으로부터 채팅 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_template(template)
# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()

# 검색 체인을 구성합니다.
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [8]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [9]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("테디의 직업은 무엇입니까?")

'테디의 직업은 개발자입니다.'

In [10]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("셜리의 직업은 무엇입니까?")

'셜리의 직업은 디자이너입니다.'

## WebBaseLoader

In [11]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.callbacks.base import BaseCallbackHandler


# Load some data to summarize
loader = WebBaseLoader("https://www.aitimes.com/news/articleView.html?idxno=131777")
docs = loader.load()
content = docs[0].page_content


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)


prompt = hub.pull("teddynote/chain-of-density-korean")

# Create the chain, including
chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-4-turbo-preview",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)


```json
[
    {
        "Missing_Entities": "",
        "Denser_Summary": "이 기사는 데이터사이언스, 머신러닝, 인공지능에 대한 개념을 설명하고 있습니다. 박정현 서울대 EPM 연구원이 작성한 이 글에서는 이 세 분야가 우리 생활에 어떻게 적용되고 있는지, 그리고 이 분야들을 공부하고자 하는 사람들이 어떻게 접근해야 하는지에 대한 정보를 제공합니다. 또한, 이 분야들의 정의와 차이점, 그리고 각 분야에서 사용되는 기술들에 대한 설명도 포함되어 있습니다. 이러한 내용을 통해 독자들은 데이터사이언스, 머신러닝, 인공지능에 대한 기본적인 이해를 할 수 있게 됩니다."
    },
    {
        "Missing_Entities": "데이터사이언스 대학원; 인공지능 국가전략; 지도학습; 비지도 학습",
        "Denser_Summary": "박정현 서울대 EPM 연구원은 데이터사이언스, 머신러닝, 인공지능의 개념과 실생활 적용 예를 설명하며, 이 분야들의 공부 방향을 제시합니다. 데이터사이언스 대학원 설립과 인공지능 국가전략 발표 등 교육 및 정책 변화를 언급하고, 머신러닝의 지도학습과 비지도 학습을 포함한 기술적 접근 방식을 설명합니다. 이 글은 이 분야에 관심 있는 이들에게 기초적인 이해와 함께 학습 및 진로 탐색에 도움을 주는 정보를 담고 있습니다."
    },
    {
        "Missing_Entities": "알파고; GPT-3; 튜링테스트; k-means",
        "Denser_Summary": "박정현 연구원은 데이터사이언스, 머신러닝, 인공지능의 정의와 적용 예제(알파고, GPT-3)를 소개하고, 데이터사이언스 대학원과 인공지능 국가전략의 중요성을 강조합니다. 머신러닝의 지도학습, 비지도 학습 방식과 튜링테스트를 통한 인공지능의 이해, k-means 같은 알고리즘 사용을 설명하며, 이 분야의 학습자와 전문가를 위한 지침을 제공합니다. 이 글은 기술적 배경과

In [12]:
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ARTICLE'], template='Article: {ARTICLE}\nYou will generate increasingly concise, entity-dense summaries of the above article. \n\nRepeat the following 2 steps 5 times. \n\nStep 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. \nStep 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. \n\nA missing entity is:\n- relevant to the main story, \n- specific yet concise (100 words or fewer), \n- novel (not in the previous summary), \n- faithful (present in the article), \n- anywhere (can be located anywhere in the article).\n\nGuidelines:\n\n- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discu

In [13]:
print("""
'Article: {ARTICLE}\nYou will generate increasingly concise, entity-dense summaries of the above article. \n\nRepeat the following 2 steps 5 times. \n\nStep 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. \nStep 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. \n\nA missing entity is:\n- relevant to the main story, \n- specific yet concise (100 words or fewer), \n- novel (not in the previous summary), \n- faithful (present in the article), \n- anywhere (can be located anywhere in the article).\n\nGuidelines:\n\n- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.\n- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.\n- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".\n- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. \n- Missing entities can appear anywhere in the new summary.\n- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. \n\nRemember, use the exact same number of words for each summary.\nAnswer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".\nUse only KOREAN language to reply.'
""")


'Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (100 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and ma